![](https://www.immigration.ca/wp-content/uploads/2018/12/Canada-Immigration_156421093.jpeg)

# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Loading Data

In [ ]:
data = pd.read_csv('../input/immigration-to-canada/immigration_Canada.csv')
data.head()

# Preparing Data

In [ ]:
data = data.rename(columns={'OdName': 'Country', 'AreaName': 'Region'}).drop(['Type','Coverage','AREA','REG','DEV','RegName','DevName'], 
                                                                                axis=1).set_index('Country')
data

In [ ]:
data = data.drop(data.index[-1])
data.tail(3)

In [ ]:
data.shape

In [ ]:
data.isna().sum().to_frame('Null Values')

# Plotting Data Distribution

In [ ]:
plt.figure(figsize=(16,5))
sns.boxplot(data=data)
plt.show()
plt.figure(figsize=(16,5))
sns.stripplot(data=data)
plt.show()
plt.figure(figsize=(16,5))
sns.violinplot(data=data)
plt.show()

# Analysis & Visualization

## Year-wise Analysis

In [ ]:
year = pd.DataFrame(data.iloc[:,2:].sum(), columns=['Total'])
year.head()

In [ ]:
plt.figure(figsize=(14,5))
plt.title('Total Number of Immigration in each year (in thousands)', fontsize=15)
plot = sns.barplot(year.index, np.round(year.Total/1000, decimals=1))
plt.bar_label(plot.containers[0], size=9)
plt.xticks(rotation='vertical')
plt.xlabel('Year', fontsize=13)
plt.ylabel('Immigration', fontsize=13)
plt.show()
max = year[year.Total == year.Total.max()]
min = year[year.Total == year.Total.min()]
print(f'Most number of immigration: {max.Total[0]}({max.index[0]})\
      \nLeast number of immigration: {min.Total[0]}({min.index[0]})')

## Country-wise Analysis

In [ ]:
country = data.iloc[:,:1]
country['Total'] = data.sum(axis=1)
country.head()

In [ ]:
plt.figure(figsize=(15,5))
sns.scatterplot(country.index, country.Total, hue=country.Region, s=100)
plt.xlabel('Country', fontsize=13)
plt.ylabel('Immigration', fontsize=13)
plt.show()

In [ ]:
country = country.sort_values('Total', ascending=False)
country

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Top Countries from where the most number of Immigration so far')
plot = sns.barplot(np.round(country.Total[:15]/1000, decimals=1), country.index[:15])
plt.bar_label(plot.containers[0], size=9)
plt.xlabel('Total immigration (in thousands)', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.show()

## Region-wise Analysis

In [ ]:
reg = country.groupby('Region').sum().sort_values('Total', ascending=False)
reg['Percentage'] = pd.DataFrame(np.round(reg.Total/reg.Total.sum()*100, decimals=1))
reg

In [ ]:
reg.Percentage.sum()

In [ ]:
reg = reg.drop(reg.index[-1])

In [ ]:
reg.Total.plot.pie(autopct='%1.2f%%',figsize=(7,7), label='Immigration', shadow=True)
plt.show()

# Plotting World Percentage

In [ ]:
country['Percentage'] = pd.DataFrame(np.round(country.Total/country.Total.sum()*100, decimals=1))
country = country.drop(['Total'], axis=1)
country.head()

In [ ]:
import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:
map_data = dict(type = 'choropleth', locations = country.index, locationmode = 'country names', 
                z = country.Percentage, text=country.Region, colorscale= 'Reds', 
                marker = dict(line = dict(color='black', width=0.5)), colorbar = {'title':'Percentage'})
layout = dict(title = 'World Percentage in Immigration', geo = dict(showframe = False, projection = {'type':'natural earth'}))
choromap = go.Figure(data=map_data, layout=layout)
iplot(choromap)

# Thank You!